In [1]:
import polars as pl
from pyod.models.knn import KNN
import matplotlib.pyplot as plt
import os

SystemError: initialization of _internal failed without raising an exception

In [30]:
!pwd

/run/media/mohamed/MyData/Development/seismic_detection/notebooks


In [31]:
data_dir = '../../seismic_data/data/'
files: list[str] = []
for root, dirs, file in os.walk(data_dir):
    for f in file:
        files.append(os.path.join(root, f))
files = sorted(files)

In [32]:
files

['../../seismic_data/data/test/mars/0001.csv',
 '../../seismic_data/data/test/mars/0002.csv',
 '../../seismic_data/data/test/mars/0007.csv',
 '../../seismic_data/data/test/mars/0011.csv',
 '../../seismic_data/data/test/mars/0017.csv',
 '../../seismic_data/data/test/mars/0032.csv',
 '../../seismic_data/data/test/mars/0033.csv',
 '../../seismic_data/data/test/mars/0034.csv',
 '../../seismic_data/data/test/mars/0041.csv',
 '../../seismic_data/data/test/moon/12_b_00006.csv',
 '../../seismic_data/data/test/moon/12_b_00007.csv',
 '../../seismic_data/data/test/moon/12_b_00014.csv',
 '../../seismic_data/data/test/moon/12_b_00016.csv',
 '../../seismic_data/data/test/moon/12_b_00018.csv',
 '../../seismic_data/data/test/moon/12_b_00020.csv',
 '../../seismic_data/data/test/moon/12_b_00021.csv',
 '../../seismic_data/data/test/moon/12_b_00026.csv',
 '../../seismic_data/data/test/moon/12_b_00027.csv',
 '../../seismic_data/data/test/moon/12_b_00028.csv',
 '../../seismic_data/data/test/moon/12_b_00029.

In [33]:
# split the data into training and testing
files_train: list[str] = [ file for file in files if 'train' in file ]
files_test: list[str] = [ file for file in files if 'test' in file ]

In [34]:
files_train

['../../seismic_data/data/train/mars/0005.csv',
 '../../seismic_data/data/train/mars/0006.csv',
 '../../seismic_data/data/train/moon/12_a_00002.csv',
 '../../seismic_data/data/train/moon/12_a_00003.csv',
 '../../seismic_data/data/train/moon/12_a_00004.csv',
 '../../seismic_data/data/train/moon/12_a_00006.csv',
 '../../seismic_data/data/train/moon/12_a_00007.csv',
 '../../seismic_data/data/train/moon/12_a_00008.csv',
 '../../seismic_data/data/train/moon/12_a_00009.csv',
 '../../seismic_data/data/train/moon/12_a_00010.csv',
 '../../seismic_data/data/train/moon/12_a_00011.csv',
 '../../seismic_data/data/train/moon/12_a_00013.csv',
 '../../seismic_data/data/train/moon/12_a_00014.csv',
 '../../seismic_data/data/train/moon/12_a_00015.csv',
 '../../seismic_data/data/train/moon/12_a_00017.csv',
 '../../seismic_data/data/train/moon/12_a_00019.csv',
 '../../seismic_data/data/train/moon/12_a_00021.csv',
 '../../seismic_data/data/train/moon/12_a_00022.csv',
 '../../seismic_data/data/train/moon/12_

In [35]:
files_test

['../../seismic_data/data/test/mars/0001.csv',
 '../../seismic_data/data/test/mars/0002.csv',
 '../../seismic_data/data/test/mars/0007.csv',
 '../../seismic_data/data/test/mars/0011.csv',
 '../../seismic_data/data/test/mars/0017.csv',
 '../../seismic_data/data/test/mars/0032.csv',
 '../../seismic_data/data/test/mars/0033.csv',
 '../../seismic_data/data/test/mars/0034.csv',
 '../../seismic_data/data/test/mars/0041.csv',
 '../../seismic_data/data/test/moon/12_b_00006.csv',
 '../../seismic_data/data/test/moon/12_b_00007.csv',
 '../../seismic_data/data/test/moon/12_b_00014.csv',
 '../../seismic_data/data/test/moon/12_b_00016.csv',
 '../../seismic_data/data/test/moon/12_b_00018.csv',
 '../../seismic_data/data/test/moon/12_b_00020.csv',
 '../../seismic_data/data/test/moon/12_b_00021.csv',
 '../../seismic_data/data/test/moon/12_b_00026.csv',
 '../../seismic_data/data/test/moon/12_b_00027.csv',
 '../../seismic_data/data/test/moon/12_b_00028.csv',
 '../../seismic_data/data/test/moon/12_b_00029.

In [36]:
# load the data
df_train = pl.read_csv(files_train[0])
df_test = pl.read_csv(files_test[0])

In [37]:
df_train["time_rel", "velocity"]

time_rel,velocity
f64,f64
0.0,0.0
0.05,0.000128
0.1,0.000703
0.15,-0.001124
0.2,-0.003985
…,…
3599.75,-0.003213
3599.8,-0.003827
3599.85,-0.00298


In [38]:
# load the model
model = KNN()

In [40]:
# train the model
model.fit(X=df_train["time_rel", "velocity"].to_numpy(), y=df_train["mq"].to_numpy())
# predict the anomaly scores
anomaly_scores = model.decision_scores_

thread '<unnamed>' panicked at /root/.cargo/git/checkouts/rust-numpy-af6c525f61fed96f/9ba9962/src/npyffi/array.rs:53:14:
Failed to access NumPy array API capsule: PyErr { type: <class 'ImportError'>, value: ImportError("cannot import name 'fastCopyAndTranspose' from 'numpy.core._multiarray_umath' (/run/media/mohamed/MyData/Development/seismic_detection/.venv/lib/python3.9/site-packages/numpy/core/_multiarray_umath.py)"), traceback: Some(<traceback object at 0x7f6f7af43440>) }


PanicException: Failed to access NumPy array API capsule: PyErr { type: <class 'ImportError'>, value: ImportError("cannot import name 'fastCopyAndTranspose' from 'numpy.core._multiarray_umath' (/run/media/mohamed/MyData/Development/seismic_detection/.venv/lib/python3.9/site-packages/numpy/core/_multiarray_umath.py)"), traceback: Some(<traceback object at 0x7f6f7af43440>) }

In [ ]:
# plot the anomaly scores
plt.plot(anomaly_scores)
plt.title("Anomaly Scores")
plt.xlabel("Data Point")
plt.ylabel("Anomaly Score")
plt.show()